In [42]:
# SUP GUYS IT'S YA GIRL LET'S DO THIS SHIT
# here I am importing things 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap  #Tom  doesn't have this. How do you install it?

%matplotlib inline

In [43]:
test = pd.read_csv('./assets/test.csv')
train = pd.read_csv('./assets/train.csv')
spray = pd.read_csv('./assets/spray.csv')
weather = pd.read_csv('./assets/weather.csv')

In [44]:
# checking out the first five rows of the test data

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [45]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


## Doing some initial analysis and cleaning on 'train' and 'test'

In [46]:
# dropping the additional index column 

test.drop(columns = 'Id', inplace = True)

In [47]:
# changing the 'Date' column to a datetime

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [48]:
# changing the date to the index

train = train.set_index('Date')
test = test.set_index('Date')

In [49]:
# confirming it worked
print(type(train.index[0]))
train.head()

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
Date,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [50]:
# wanted to see how the counts of mosquitos and the infected mosquito counts
# related

train.sort_values('WnvPresent', ascending = False).head(20)

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
Date,,,,,,,,,,,
2007-09-05,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,13,1
2011-08-12,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,8,1
2013-08-22,"3700 South Pulaski Road, Chicago, IL 60623, USA",CULEX PIPIENS/RESTUANS,37,S PULASKI RD,T063,"3700 S PULASKI RD, Chicago, IL",41.825610,-87.726549,9,6,1
2013-08-22,"5000 South Union Avenue, Chicago, IL 60609, USA",CULEX PIPIENS/RESTUANS,50,S UNION AVE,T082,"5000 S UNION AVE, Chicago, IL",41.803423,-87.642984,8,15,1
2007-08-21,"5200 South Nordica Avenue, Chicago, IL 60638, USA",CULEX PIPIENS,52,S NORDICA,T070,"5200 S NORDICA, Chicago, IL",41.797616,-87.797894,8,10,1
2013-08-22,"6700 South Kedzie Avenue, Chicago, IL 60629, USA",CULEX PIPIENS,67,S KEDZIE AVE,T066,"6700 S KEDZIE AVE, Chicago, IL",41.771660,-87.703047,8,14,1
2013-08-22,"6700 South Kedzie Avenue, Chicago, IL 60629, USA",CULEX RESTUANS,67,S KEDZIE AVE,T066,"6700 S KEDZIE AVE, Chicago, IL",41.771660,-87.703047,8,4,1
2007-08-21,"5200 South Nordica Avenue, Chicago, IL 60638, USA",CULEX PIPIENS/RESTUANS,52,S NORDICA,T070,"5200 S NORDICA, Chicago, IL",41.797616,-87.797894,8,13,1
2011-08-12,"5100 North Mont Clare Avenue, Chicago, IL 6065...",CULEX RESTUANS,51,N MONT CLARE AVE,T223,"5100 N MONT CLARE AVE, Chicago, IL",41.973845,-87.805059,9,3,1


In [51]:
# created an additional column that shows the percentage of infected mosquitos

train['PercentageWnv'] = train['WnvPresent'] / train['NumMosquitos']

In [52]:
# make sure it works
train['PercentageWnv'].value_counts()

0.000000    9955
0.020000     108
1.000000      20
0.076923      20
0.200000      19
0.111111      17
0.062500      15
0.333333      15
0.142857      13
0.047619      12
0.250000      12
0.083333      12
0.025641      11
0.100000      11
0.027027      11
0.066667      11
0.032258      11
0.125000      11
0.055556      11
0.037037      10
0.166667      10
0.043478      10
0.058824       9
0.050000       9
0.090909       8
0.041667       8
0.071429       8
0.035714       8
0.027778       8
0.026316       8
0.020833       8
0.500000       7
0.030303       7
0.033333       7
0.045455       6
0.023256       6
0.021277       6
0.052632       6
0.038462       6
0.034483       6
0.022222       5
0.024390       5
0.020408       5
0.021739       5
0.040000       5
0.023810       5
0.025000       5
0.028571       5
0.031250       4
0.029412       3
0.022727       3
Name: PercentageWnv, dtype: int64

In [53]:
train.head()

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,PercentageWnv
Date,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0.0
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0.0
2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,0.0
2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,0.0
2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,0.0


### Now to the "Spray" data

In [54]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [55]:
spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

In [56]:
# decided to put the most common time in the null time values

spray['Time'].value_counts()

7:44:32 PM     541
8:57:56 PM       5
8:57:46 PM       5
8:58:26 PM       5
8:55:46 PM       5
8:55:36 PM       5
8:55:26 PM       5
8:59:06 PM       5
9:05:56 PM       5
9:37:27 PM       5
9:40:27 PM       5
8:58:56 PM       5
9:35:47 PM       5
9:38:27 PM       5
9:31:27 PM       5
9:07:45 PM       4
9:31:17 PM       4
8:28:21 PM       4
9:40:07 PM       4
9:44:07 PM       4
9:13:46 PM       4
9:36:27 PM       4
9:45:15 PM       4
8:24:41 PM       4
9:25:26 PM       4
9:16:05 PM       4
8:59:36 PM       4
9:08:25 PM       4
8:59:46 PM       4
8:44:05 PM       4
              ... 
9:49:41 PM       1
9:36:36 PM       1
9:44:38 PM       1
10:09:57 PM      1
11:51:10 PM      1
9:18:08 PM       1
8:51:07 PM       1
9:32:38 PM       1
8:37:12 PM       1
9:16:31 PM       1
11:44:26 PM      1
8:09:03 PM       1
10:02:27 PM      1
10:36:21 PM      1
10:46:41 PM      1
9:40:35 PM       1
10:22:48 PM      1
11:53:01 PM      1
10:03:07 PM      1
11:50:01 PM      1
9:24:00 PM       1
7:56:10 PM  

In [57]:
# fill in na values with the most common time

spray['Time'].fillna('7:44:32 PM', inplace = True)

In [58]:
# combine date and time columns 

spray['DateTime'] = spray['Date'] + ' ' + spray['Time']

In [59]:
# convert to datetime and convert to index

spray['DateTime'] = pd.to_datetime(spray['DateTime'])
spray = spray.set_index('DateTime')

In [60]:
# drop date and time columns as they are redundant 

spray.drop(columns = ['Date', 'Time'], inplace = True)

In [61]:
# check to make sure it worked

spray.head()

,Latitude,Longitude
DateTime,,
2011-08-29 18:56:58,42.391623,-88.089163
2011-08-29 18:57:08,42.391348,-88.089163
2011-08-29 18:57:18,42.391022,-88.089157
2011-08-29 18:57:28,42.390637,-88.089158
2011-08-29 18:57:38,42.390410,-88.088858


In [62]:
# THIS IS A TEST AND IS NOT GOOD, TK WAS WORKING ON BASEMAP AS WELL. LEAVING THIS IN INCASE ANYONE
# WANTS TO WORK ON IT

# plt.figure(figsize=(8, 8))
# earth = Basemap(projection = 'merc',
#                 lon_0 = -87.6298,
#                 lat_0 = 41.8781,
#                 llcrnrlon=-85, 
#                 llcrnrlat=39,
#                 urcrnrlon=-89, 
#                 urcrnrlat=45,
#                 resolution = 'h'
#                )
# earth.bluemarble(alpha=0.42)
# earth.drawcoastlines(color='grey', linewidth=1)
# #earth.drawcounties(color = 'grey', linewidth=1)
# plt.scatter(spray['Longitude'], spray['Latitude'], 
#             c='red',alpha=.5)
# #plt.figsize()
# plt.xlabel("Test");

### moving to 'Weather' data

In [63]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [64]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [65]:
# converting date to datetime

weather['Date'] = pd.to_datetime(weather['Date'])

In [66]:
# setting date to index

weather = weather.set_index('Date')

In [67]:
# station 2 sucks at recording things, it has a lot more missing data.

weather.head()

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,1,83,50,67,14,51,56,0,2,0448,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
2007-05-01,2,84,52,68,M,51,57,0,3,-,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2007-05-02,1,59,42,51,-3,42,47,14,0,0447,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
2007-05-02,2,60,43,52,M,42,47,13,0,-,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
2007-05-03,1,66,46,56,2,40,48,9,0,0446,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [68]:
# here we will replace 'M' with nan values (as it is missing) and change 'T' to a small decimal (as it stands for trace)
# as it was recorded as 'trace'

weather.replace('M', np.nan, inplace = True)
weather.replace('  T', 0.005, inplace = True) #This was annoying

In [70]:
weather.head(10) # Checking to make sure it worked. There are a few NANs and .005s in there.

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,1,83,50,67,14,51,56,0,2,0448,...,,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2
2007-05-01,2,84,52,68,NaN,51,57,0,3,-,...,,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6
2007-05-02,1,59,42,51,-3,42,47,14,0,0447,...,BR,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4
2007-05-02,2,60,43,52,NaN,42,47,13,0,-,...,BR HZ,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4
2007-05-03,1,66,46,56,2,40,48,9,0,0446,...,,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9
2007-05-03,2,67,48,58,NaN,40,50,7,0,-,...,HZ,NaN,NaN,NaN,0.00,29.46,30.12,12.9,6,13.2
2007-05-04,1,66,49,58,4,41,50,7,0,0444,...,RA,0,NaN,0.0,0.005,29.31,30.05,10.4,8,10.8
2007-05-04,2,78,51,NaN,NaN,42,50,NaN,NaN,-,...,,NaN,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4
2007-05-05,1,66,53,60,5,38,49,5,0,0443,...,,0,NaN,0.0,0.005,29.40,30.10,11.7,7,12.0


In [38]:
weather.isnull().sum()

Station           0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise           0
Sunset            0
CodeSum           0
Depth          1472
Water1         2944
SnowFall       1472
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [71]:
# deleting the columns that give little to no information

weather.drop(columns = ['Water1', 'Depth', 'SnowFall', 'Depart', 'CodeSum'], inplace = True)

In [73]:
# here we are filling in the missing 'Tavg' data with the average 

weather['Tavg'].fillna(((weather['Tmax'] + weather['Tmin']) / 2), inplace = True)

In [74]:
# making the columns with null values easier to read
# What null columns are left to deal with

null_columns = weather.columns[weather.isnull().any()]
weather[null_columns].isnull().sum()

WetBulb         4
Heat           11
Cool           11
PrecipTotal     2
StnPressure     4
SeaLevel        9
AvgSpeed        3
dtype: int64

In [77]:
print('Null Rows: ', weather[weather.isnull().any(axis=1)][null_columns].shape[0])
print('Total Rows: ', weather.shape[0])
#Tom G thinks we should just drop these

Null Rows:  26
Total Rows:  2944


In [79]:
#Being overly cautious before dropping data
weather.dropna().shape

(2918, 16)

In [80]:
weather.dropna(inplace=True)

### i, emma york, want to drop the heat and cool columns. can we proceed with this?

Since there are no nulls anymore, I (Tom G) say we might as well leave them in and then if we don't want to use them in our model we can set our X in the modeling notebook

In [81]:
# weather.drop(columns = ['Heat', 'Cool'], inplace = True)

In [110]:
weather.head(2)

,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,
2007-05-01,1,83,50,67,51,56,0,2,0448,1849,0.00,29.10,29.82,1.7,27,9.2
2007-05-01,2,84,52,68,51,57,0,3,-,-,0.00,29.18,29.82,2.7,25,9.6


## Set the '-' from weather station 2 to the times from weather station 1


In [140]:
print(weather[weather['Station']==1].shape[0])
print(weather[weather['Station']==2].shape[0]) #This is annoying. I was hoping that there would be the same amount in both so I could change the sunrise/sunset for station 2 easier.

1464
1454


In [162]:
# There is definitely a prettier way of doing this. 
# Filling in t
for i in range(weather.shape[0]-1):
    if weather.index[i] == weather.index[i+1]:
        if weather.iloc[i, 0] == 1: #Station 1
            if weather.iloc[i+1, 0] == 2: #Station 2
                weather.iloc[i+1, 8:10] = weather.iloc[i, 8:10]
            

In [163]:
weather.head(6) #checking to make sure that they worked correctly

,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,
2007-05-01,1,83,50,67,51,56,0,2,0448,1849,0.00,29.10,29.82,1.7,27,9.2
2007-05-01,2,84,52,68,51,57,0,3,0448,1849,0.00,29.18,29.82,2.7,25,9.6
2007-05-02,1,59,42,51,42,47,14,0,0447,1850,0.00,29.38,30.09,13.0,4,13.4
2007-05-02,2,60,43,52,42,47,13,0,0447,1850,0.00,29.44,30.08,13.3,2,13.4
2007-05-03,1,66,46,56,40,48,9,0,0446,1851,0.00,29.39,30.12,11.7,7,11.9
2007-05-03,2,67,48,58,40,50,7,0,0446,1851,0.00,29.46,30.12,12.9,6,13.2


In [172]:
ind_dash = weather[weather['Sunrise']=='-'].index
weather.loc[ind_dash, :] 
#These 7 rows do not have metrics from station 1 to change their Sunrise/Sunset to

,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,
2009-06-18,2,81,63,72,64,67,0,7,-,-,0.110,29.15,29.79,3.7,17,5.8
2009-06-26,2,86,72,79,61,67,0,14,-,-,0.000,29.20,29.83,6.4,4,8.0
2009-09-07,2,77,63,70,59,63,0,5,-,-,0.000,29.44,30.09,6.3,4,6.9
2011-09-08,2,74,62,68,54,59,0,3,-,-,0.060,29.36,30.03,14.9,2,15.2
2011-09-20,2,75,53,64,54,58,1,0,-,-,0.010,29.33,29.99,5.8,17,6.1
2012-09-03,2,87,73,80,69,73,0,15,-,-,0.000,29.23,29.87,5.0,7,5.9
2013-08-11,2,84,63,74,57,64,0,9,-,-,0.005,29.42,30.06,4.0,24,5.4


In [168]:
weather.loc['2009-06-18', :] 
#Checking to make sure that there aren't any other data points for the dates listed above.

Station            2
Tmax              81
Tmin              63
Tavg              72
DewPoint          64
WetBulb           67
Heat               0
Cool               7
Sunrise            -
Sunset             -
PrecipTotal     0.11
StnPressure    29.15
SeaLevel       29.79
ResultSpeed      3.7
ResultDir         17
AvgSpeed         5.8
Name: 2009-06-18 00:00:00, dtype: object

There are only 7, Tom G is okay with dropping them

In [175]:
print(weather.shape[0]-len(ind_dash))

2911


In [176]:
weather.drop(index=ind_dash).shape

(2911, 16)

In [177]:
weather.drop(index=ind_dash, inplace=True)

## Setting the columns to their correct types


In [86]:
train.dtypes

Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
PercentageWnv             float64
dtype: object

In [87]:
train.head(1) # These dtypes look right

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,PercentageWnv
Date,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0.0


In [89]:
test.dtypes #these types look right

Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object

In [109]:
weather.dtypes

Station          int64
Tmax             int64
Tmin             int64
Tavg            object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [108]:

weather.head()

,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,
2007-05-01,1,83,50,67,51,56,0,2,0448,1849,0.00,29.10,29.82,1.7,27,9.2
2007-05-01,2,84,52,68,51,57,0,3,-,-,0.00,29.18,29.82,2.7,25,9.6
2007-05-02,1,59,42,51,42,47,14,0,0447,1850,0.00,29.38,30.09,13.0,4,13.4
2007-05-02,2,60,43,52,42,47,13,0,-,-,0.00,29.44,30.08,13.3,2,13.4
2007-05-03,1,66,46,56,40,48,9,0,0446,1851,0.00,29.39,30.12,11.7,7,11.9


In [107]:
objs = []
for col in weather.columns:
    if weather[col].dtypes == 'O':
        objs.append(col)
        
weather[objs].dtypes

Tavg           object
WetBulb        object
Heat           object
Cool           object
Sunrise        object
Sunset         object
PrecipTotal    object
StnPressure    object
SeaLevel       object
AvgSpeed       object
dtype: object

In [178]:
floats = ['PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed']
ints = objs[0:6]
weather[floats] = weather[floats].astype(float)
weather[ints] = weather[ints].astype(int)
#print(floats)
#print(ints)

In [180]:
weather.dtypes

Station          int64
Tmax             int64
Tmin             int64
Tavg             int64
DewPoint         int64
WetBulb          int64
Heat             int64
Cool             int64
Sunrise          int64
Sunset           int64
PrecipTotal    float64
StnPressure    float64
SeaLevel       float64
ResultSpeed    float64
ResultDir        int64
AvgSpeed       float64
dtype: object

Spray

In [184]:
spray.dtypes

Latitude     float64
Longitude    float64
dtype: object

In [199]:
test.isnull().sum().sum()

0

In [200]:
train.isnull().sum().sum()

0

In [201]:
weather.isnull().sum().sum()

0

In [202]:
spray.isnull().sum().sum()

0

### Splitting up the two weather stations just in case we want to use them individually later

In [181]:
weather_station_1 = weather.loc[weather['Station'] == 1]
weather_station_2 = weather.loc[weather['Station'] == 2]

In [182]:
weather_station_1.shape[0] + weather_station_2.shape[0] == weather.shape[0]

True

## Save
Save the cleaned files as new csvs to use in other notebooks.

In [183]:
train.to_csv('./assets/train_cleaned.csv')
test.to_csv('./assets/test_cleaned.csv')
spray.to_csv('./assets/spray_cleaned.csv')
weather.to_csv('./assets/weather_cleaned.csv')
weather_station_1.to_csv('./assets/weather_station_1_cleaned.csv')
weather_station_2.to_csv('./assets/weather_station_2_cleaned.csv')